# Introduction to Ray
Ray is a open-source unified framework for scaling AI and Python applications like machine learning. It provides the compute layer for parallel processing so that you don't need to be a dsitributed systems expert.

In [ ]:
# suppressing all error messages
import warnings
warnings.filterwarnings('ignore')

## 1 | Paralleizing Tasks with Ray
We can turn functions and classes eaily into Ray tasks and actors, for Python with simple primitives for building and running distributed applications. 

### Simplified Explanation
Ray make it easy to run functions and classes on multiple computers at the same time. It handles the complex parts of managing tasks and resources, so you can focus on building scalable applications with simple commands

In [ ]:
# importing ray
import ray

In [ ]:
# initializing ray
ray.init()

In [ ]:
# the @ is a decorator that tells Ray to treat function f as a "remote task" that can be executed in parallel
@ray.remote
def f(x):
    return x * x

In [ ]:
# creates a list of futures, which are placeholders for results of remote tasks
futures = [f.remote(i) for i in range(4)]

# used to retrieve the results of the futures
print(ray.get(futures))

In [ ]:
# stopping ray
ray.shutdown()

## 2 | Counting Digits of PI
Sometimes we just want to do something simple in parallel. Ray is useful for simpler, repetitive tasks that need to be run multiple times. The following example below is about processing 100,000 time series. Each time series needs to be processed using the same algorithm.

Instead of processing them one by one, Ray can handle the tasks in parallel, so multiple time series can be processed at the same time, which speeds up the overall work.

### The Pi Example
We take the the simple example of counting digits of Pi. The algorithm works by generating random x and y, and if x^2 + y^2 < 1, it's inside the circle, we count as in. 

In [ ]:
# all the necessary imports
import ray
import random
import time
import math
from tqdm import tqdm
from fractions import Fraction

In [ ]:
# starting up ray
ray.init(address='auto')

In [ ]:
@ray.remote
def pi4_sample(sample_count):
    in_count = 0
    for i in range(sample_count):
        x = random.random()
        y = random.random()
        if x*x + y*y <= 1:
            in_count += 1
    return Fraction(in_count, sample_count)

In [ ]:
SAMPLE_COUNT = 1000 * 1000
start = time.time()
future = pi4_sample.remote(sample_count=SAMPLE_COUNT)
pi4 = ray.get(future)
end = time.time()
duration = end - start
print(f'Running {SAMPLE_COUNT} tests took {duration} seconds')

In [ ]:
FULL_SAMPLE_COUNT = 1000 * 1000 * 1000 
BATCHES = int(FULL_SAMPLE_COUNT / SAMPLE_COUNT)
print(f'Doing {BATCHES} batches')
results = []
for _ in range(BATCHES):
    results.append(pi4_sample.remote(sample_count=SAMPLE_COUNT))
output = ray.get(results)